In [32]:
# required_rows = []
# scan_numbers = set()
# current_peptides = []
# with open("../data.tsv", "r") as tsv_file:
#     reader = csv.reader(tsv_file, delimiter='\t')
#     header = next(reader)  # Skip the header row
#     for row in reader:
#         filepath = row[-1]
#         peptide = row[9]
#         scan_number = int(row[2])
#         if filepath in train_mzml_files:
#             if scan_number not in scan_numbers:
#                 current_peptides = []
#                 current_peptides.append(peptide)
#                 count = 1
#                 required_rows.append(row)
#                 scan_numbers.add(scan_number)
#             elif scan_number in scan_numbers:
#                 if peptide in current_peptides:
#                     continue
#                 elif count < 5:
#                     count += 1
#                     continue
#                 elif count >= 5 and count < 8:
#                     count += 1
#                     required_rows.append(row)
#                     current_peptides.append(peptide)
#                 elif count >= 8 and len(current_peptides) < 4 and peptide not in current_peptides:
#                     count += 1
#                     required_rows.append(row)
#                     current_peptides.append(peptide)
#                 else:
#                     continue
        
# with open("train.csv", "w", newline='') as train_file:
#     writer = csv.writer(train_file)
#     writer.writerow(header)
#     writer.writerows(required_rows)



In [30]:
def get_unique_mzml_files(filename):
    unique_mzml_files = set()
    with open(filename, "r") as tsv_file:
        reader = csv.reader(tsv_file, delimiter='\t')
        header = next(reader)  # Skip the header row
        for row in reader:
            filepath = row[-1]
            if filepath not in unique_mzml_files:
                unique_mzml_files.add(filepath)
            if len(unique_mzml_files) >= 2:
                break
    return list(unique_mzml_files)

train_mzml_files = get_unique_mzml_files("./data/raw/train_data.tsv")
test_mzml_files = get_unique_mzml_files("./data/raw/test_data.tsv")

In [31]:
train_mzml_files

['MSV000083508/ccms_peak/lung/Trypsin_HCD_QExactiveplus/01279_B03_P013163_B00_N18_R1.mzML',
 'MSV000083508/ccms_peak/lung/Trypsin_HCD_QExactiveplus/01088_A05_P010740_S00_N33_R1.mzML']

In [41]:
import csv

def generate_tsv(unique_mzml_files, data_filename):
    required_rows = []
    scan_numbers = set()
    current_peptides = []
    with open(data_filename, "r") as tsv_file:
        reader = csv.reader(tsv_file, delimiter='\t')
        header = next(reader)  # Skip the header row
        for row in reader:
            filepath = row[-1]
            peptide = row[9]
            scan_number = int(row[2])
            if filepath in unique_mzml_files:
                if scan_number not in scan_numbers:
                    current_peptides = []
                    current_peptides.append(peptide)
                    count = 1
                    required_rows.append(row)
                    scan_numbers.add(scan_number)
                elif scan_number in scan_numbers:
                    if peptide in current_peptides:
                        continue
                    elif count < 5:
                        count += 1
                        continue
                    elif count >= 5 and count < 8:
                        count += 1
                        required_rows.append(row)
                        current_peptides.append(peptide)
                    elif count >= 8 and len(current_peptides) < 4 and peptide not in current_peptides:
                        count += 1
                        required_rows.append(row)
                        current_peptides.append(peptide)
                    else:
                        continue
    return header, required_rows

In [ ]:
header, train_rows = generate_tsv(train_mzml_files, "./data/raw/train_data.tsv")
header, test_rows = generate_tsv(test_mzml_files, "./data/raw/test_data.tsv")

In [26]:
def write_tsv(header, required_rows, filename):
    with open(filename, "w") as tsv_file:
        writer = csv.writer(tsv_file, delimiter='\t', lineterminator='\n')
        writer.writerow(header)
        writer.writerows(required_rows)


In [37]:
write_tsv(header, train_rows, "train.tsv")

In [40]:
write_tsv(header, test_rows, "test.tsv")